## Integrace funkce jedné proměnné


V tomto cvičení si nejprve vyzkoušíme nalézt primitivní funkci k zadané funkci pomocí symbolické manipulace a následně si vyzkoušíme spočítat určitý integrál pomocí lichoběžníkového pravidla z Newton-Cotesových vzorců.

#### CACv.11.1: Neurčitý integrál

Nalezněte primitivní funkci k zadané funkci (tzn. řešte neurčitý integrál zadané funkce).

<img src="MS11_soubory/casymbol.png" alt="Drawing" style="width: 270px;"/>

Pro výpočet neurčitého integrálu využijeme modul sympy, který není součástí standardní knihovny jazyka Python3 a je nutné ho instalovat přes balíčkovací systém pip.

Manuál: [Sympy](https://docs.sympy.org/latest/modules/integrals/integrals.html)

In [ ]:
!python3 -m pip install sympy
import sympy

In [1]:
import sympy
x = sympy.Symbol("x")
sympy.integrate(-3*x**2 + 4*x + 5, x)

-x**3 + 2*x**2 + 5*x

#### CACv.11.2: Určitý integrál

Spočítejte následující určitý integrál na zadaném intervalu.

<img src="MS11_soubory/caintegral.png" alt="Drawing" style="width: 170px;"/>

Výpočet provedeme pomocí lichoběžníkové integrace. Pro procvičení provedeme integraci nejprve naivním způsobem, následně ve formě Newton-Cotesovo vzorce a na závěr pomocí knihovny numpy.

In [20]:
def f(x): 
    return -3*x**2 + 4*x + 5
   
a = 0
b = 3
dx = 0.1

 #analytické řešení: I(-3x2 + 4x + 5)(a,b) = [-x3 + 2x2 + 5x](0,3) = -27 + 18 + 15 = 6

integrace lichoběžníkovým pravidlem

<img src="MS11_soubory/trap.png" alt="Drawing" style="width: 400px;"/>

In [10]:
integral = 0
x = a
i = 0
while x < b:
    integral += dx * (f(x) + f(x+dx))/2
    x += dx
print(integral)

5.984999999999994


integrace ve formě Newtonových-Cotesových vzorců

<img src="MS11_soubory/nctrap.png" alt="Drawing" style="width: 270px;"/>

In [11]:
n = int((b-a)//dx)+1                # počet dělení
integral = f(a) + f(b)              # násobení h/2 až na konci
for i in range(1, n):
    integral += 2*f(a+i*dx)
integral *= dx/2
print(integral)

5.984999999999998


pomocí knihovny numpy

In [ ]:
!python3 -m pip install numpy

In [12]:
import numpy as np
x = np.arange(a, b+dx, dx)
y = f(x)
print(np.trapz(y, dx=dx))               # trapezoid - lichoběžník

5.984999999999999


pomocí knihovny scipy

In [ ]:
!python3 -m pip install scipy

In [ ]:
# pro apple silicon architektury
!python3 -m pip install --pre -i https://pypi.anaconda.org/scipy-wheels-nightly/simple scipy

In [14]:
from scipy import integrate

In [15]:
# trapezoid
print(integrate.trapezoid(y, x=x))          # první argument vždy funkce ve formě pole, druhý pole x
print(integrate.trapezoid(y, dx=dx))        # druhý argument šířka vzorkování

5.985
5.984999999999999


In [16]:
# simpson
x = np.arange(a, b+dx, dx)
y = f(x)
print(integrate.simpson(y=y, x=x))          # argumenty stejné jako trapezoid
print(integrate.simpson(y=y, dx=dx))

6.000000000000002
5.999999999999996


In [19]:
# romberg
print(integrate.romberg(f, a, b))           # funkce, ne pole

6.0


In [81]:
# gaussian
print(integrate.quadrature(f, a, b))

(5.9999999999999964, 1.1546319456101628e-14)


### Samostatná cvičení

#### SCv.11.1: Symbolická matematika

Pomocí symbolické matematiky vypočítejte následující integrály. Zkuste předem odhadnout podmínky integrace a existence primitivní funkce.

<img src="MS11_soubory/sym1.png" alt="Drawing" style="width: 150px;"/>

In [21]:
import math
import numpy as np
x = sympy.Symbol("x")
a = sympy.Symbol("a")
f = sympy.exp(-a*x**2)
#sympy.Integral(f, x).doit()        # pouze výpis, výpočet přes doit()
sympy.integrate(f, x)
#sympy.integrate(f, (x, -np.inf, np.inf))

Piecewise((sqrt(pi)*erf(sqrt(a)*x)/(2*sqrt(a)), Ne(a, 0)), (x, True))

<img src="MS11_soubory/sym2.png" alt="Drawing" style="width: 150px;"/>

In [22]:
import math
x = sympy.Symbol("x")
a = sympy.Symbol("a")
sympy.integrate(x**a, x)

Piecewise((x**(a + 1)/(a + 1), Ne(a, -1)), (log(x), True))

<img src="MS11_soubory/sym3.png" alt="Drawing" style="width: 200px;"/>

In [23]:
import math
x = sympy.Symbol("x")
sympy.integrate(sympy.acos(sympy.sin(x)), x)

Integral(acos(sin(x)), x)

<img src="MS11_soubory/sym4.png" alt="Drawing" style="width: 200px;"/>

In [43]:
import math
x = sympy.Symbol("x")
sympy.integrate(sympy.log(sympy.sin(x)), x)

Integral(log(sin(x)), x)

#### SCv.11.2: Určitý integrál

Pomocí built-in funkcí nebo metod numerické matematiky vypočítejte následující určité integrály. Použijte alespoň tři různé integrační metody a porovnejte je mezi sebou z hlediska přesnosti výpočtu.

<img src="MS11_soubory/uint1.png" alt="Drawing" style="width: 120px;"/>

In [24]:
import numpy as np

def f(x):
    return np.sin(x)

a = 0
b = 2*np.pi
dx = 0.1

x = np.arange(a, b+dx, dx)
y = f(x)

print("trapezoid: ", integrate.trapezoid(y, x=x))
print("simpson: ", integrate.simpson(y=y, x=x))
print("romberg: ", integrate.romberg(f, a, b))

trapezoid:  0.00014124579393251947
simpson:  0.00014066340514081377
romberg:  2.5648942582957195e-16


<img src="MS11_soubory/uint2.png" alt="Drawing" style="width: 150px;"/>

In [25]:
import numpy as np

def f(x):
    return x**2 - 2*x + 6

a = 0
b = 1
dx = 0.1

x = np.arange(a, b+dx, dx)
y = f(x)

print("trapezoid: ", integrate.trapezoid(y, x=x))
print("simpson: ", integrate.simpson(y=y, x=x))
print("romberg: ", integrate.romberg(f, a, b))

trapezoid:  5.335
simpson:  5.333333333333332
romberg:  5.333333333333333


<img src="MS11_soubory/uint3.png" alt="Drawing" style="width: 160px;"/>

In [26]:
import numpy as np

def f(x):
    return np.exp(3*x)*np.sin(2*x)

a = 0
b = np.pi/4.
dx = 0.1

x = np.arange(a, b+dx, dx)
y = f(x)

print("trapezoid: ", integrate.trapezoid(y, x=x))
print("simpson: ", integrate.simpson(y=y, x=x))
print("romberg: ", integrate.romberg(f, a, b))

trapezoid:  2.771455172001907
simpson:  2.7460056600827754
romberg:  2.5886286325075183


### Domácí cvičení

#### DCv.11.1: Newtonovy-Cotesovy vzorce
Naprogramujte si zbylé Newtonovy-Cotesovy vzorce (Simpsonovo pravidlo, Simpsonovo 3/8 pravidlo, Booleovo pravidlo) svépomocí a porovnejte jejich přesnost s lichoběžníkovým pravidlem. 

<img src="MS11_soubory/nc.png" alt="Drawing" style="width: 600px;"/>

Kolik subintervalů potřebují jednotlivé metody, aby dosáhly dostatečné přenosti?

#### DCv.11.2: Interpolace a integrace

Následující tabulka obsahuje souřadnice hranice pozemku v metrech. Pomocí lichoběžníkové integrace zjistěte rozlohu pozemku v metrech čtverečních. Data také naleznete v souboru pozemek.txt.

<table>
    <thead style="background-color:#444444;">
        <td>x</td><td>y</td>
    </thead>
    <tr>
        <td>0</td><td>125</td>
    </tr>
    <tr>
        <td>100</td><td>125</td>
    </tr>
    <tr>
        <td>200</td><td>120</td>
    </tr>
    <tr>
        <td>300</td><td>112</td>
    </tr>
    <tr>
        <td>400</td><td>90</td>
    </tr>
    <tr>
        <td>500</td><td>90</td>
    </tr>
    <tr>
        <td>600</td><td>95</td>
    </tr>
    <tr>
        <td>700</td><td>88</td>
    </tr>
    <tr>
        <td>800</td><td>75</td>
    </tr>
    <tr>
        <td>900</td><td>35</td>
    </tr>
    <tr>
        <td>1000</td><td>0</td>
    </tr>

</table>

#### DCv.11.3: Rombergova integrace

Jeden ze způsobů jak zvětšit přenost intergrace je Richardsonova extrapolace (RE). Cílem RE je zlepšit integrační přenost eliminací chyb. Rombergova integrace (také známo jako Rombergova kvadratura) je metoda, která zlepšuje výsledek lichoběžníkové integrace odstraněním chyb Richardsonovou extrapolací. Pochopení metody vyžaduje mentální uchopení konceptu rekurze. Pročtěte si následující zdroje a projděte si kód s implementovanou Rombergovou integrací, který jsme pro vás připravili.

Zdroje k samostudiu: [CZ](http://physics.ujep.cz/~mlisal/nm_1/jskvor/PDF/IntegralRomberg.pdf) [EN](https://towardsdatascience.com/numerical-integration-romberg-integration-3f54002ab538)


$$𝑆(𝑛,𝑚)=𝑆(𝑛,𝑚−1)+(𝑆(𝑛,𝑚−1)−𝑆(𝑛−1,𝑚−1))/(4^𝑚−1)$$


In [31]:
def romberg(a, b, i, j):
    if j == 0:
        n = 2**i
        dx = (b-a)/n
        return np.trapz(f(np.linspace(a, b, n)), dx=dx)
    if 0 < j <= i:
        return romberg(a, b, i, j-1) + (romberg(a, b, i, j-1) - romberg(a, b, i-1, j-1))/(4**j - 1)

print(romberg(a, b, i=10, j=5))

2.5870882254694396
